In [11]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import import_ipynb
from CandidateExtraction import *
import os
from keras.models import load_model
import json
from Tesseract import *
from keras.models import load_model
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore")
import time
import sys
import psutil
from tqdm import tqdm

In [12]:
# dataDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")
# os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data")


In [5]:
def getNeighbourInfo(candidates, data):
    file = open("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Models/wordEmbeddings.json")
    dict = json.load(file)

    for df in candidates:
        for col in dict.keys():
            df[col] = 0
            df[col+"_X"] = 0
            df[col+"_Y"] = 0

        for index in df.index:
            Neighbours = []
            X = df["x1"][index] + df["x2"][index]
            Y = df["y1"][index] + df["y2"][index]
            X/=2
            Y/=2
            for idx in data.index:
                text = data["text"][idx].lower()
                X2 = data["X"][idx]
                Y2 = data["Y"][idx]
                if Y-Y2 >= 0 and Y-Y2 <= 0.1  and X2<X and text in dict.keys():
                    df[text][index] = dict[text]
                    df[text+"_X"][index] = X-X2
                    df[text+"_Y"][index] = Y-Y2

In [6]:
def createCandidates(file):

    data, candidateInfo = get_candidates(file)

    data = data[["text", "X", "Y"]]


    getNeighbourInfo(candidateInfo, data)

    return candidateInfo
    



In [9]:
# dfs = createCandidates("Images/20.jpeg")

In [10]:
# dfs[1]

In [7]:
def benchmark():
    
    total_steps = 1000
    progress_bar = tqdm(total=total_steps, ncols=80)
    
    n = input("Enter number of iterations : ")
    
    
    start = time.time()
    for i in range(int(n)):
        dfs = createCandidates("Images/20.jpeg")
    end = time.time()
    
    inferenceTime = end-start
    
    cpuUsage = []
    memoryUsage = []
    
    start_time = time.time()
    step=1
    for run in range(5):
        cpuList = []
        memoryList = []
        process = psutil.Process()

        for i in range(5):
            cpuPercent = process.cpu_percent(interval=0.01)
            memory = process.memory_info().rss
            cpuList.append(cpuPercent)
            memoryList.append(memory)

            for j in range(int(n)):
                dfs = createCandidates("Images/20.jpeg")
                
                progress_bar.update(1)
                elapsed_time = time.time() - start_time
                remaining_time = elapsed_time / (step + 1) * (total_steps - step - 1)
                progress_bar.set_postfix({'ETA': progress_bar.format_interval(remaining_time)})
                step+=1
                
        avgCPU = sum(cpuList) / len(cpuList)
        cpuUsage.append(avgCPU)
        avgMemory = sum(memoryList) / len(memoryList)
        memoryUsage.append(avgMemory)
    
    maxCPU = max(cpuUsage)
    minCPU = min(cpuUsage)
    
    maxMemory = max(memoryUsage)
    minMemory = min(memoryUsage)
    
    progress_bar.close()
    
    
    return inferenceTime, maxCPU, minCPU, maxMemory, minMemory
    
    

In [8]:
# inferenceTime, maxCPU, minCPU, maxMemory, minMemory = benchmark()

In [9]:
# print("BENCHMARK FOR 3 FIELDS \n\n")
# print("AVERAGE TIME TAKEN : ", "{:.2f}".format(inferenceTime), "sec")
# print("MAX CPU USED: ", "{:.2f}".format(maxCPU), "%")
# print("MAX MEMORY USED: ", maxMemory//(1024*1024), "MB")

In [1]:
# os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
# imagesDir = os.listdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")
# total = 110
# for file in imagesDir:
#     print(total, " Left")
#     total-=1
#     dfs = createCandidates(file)
#     os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceNumberDataset")
#     dfs[0].to_csv(file + ".csv")
#     os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/InvoiceDateDataset")
#     dfs[1].to_csv(file + ".csv")
#     os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/TotalAmountDataset")
#     dfs[2].to_csv(file + ".csv")
#     os.chdir("/home/aman/Documents/Tally/Git-Document-AI/Document-AI/Google'sRepresentationLearning/Data/Images")